# Import Library

In [1]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import zipfile

# Membuat Folder

In [2]:
# Fungsi untuk membuat folder jika belum ada
def create_path(destination):
    if not os.path.exists(destination):
        os.mkdir(destination)
    print(f"Output disimpan di {destination}")
    return destination

Fungsi `create_path` memastikan bahwa folder tempat penyimpanan file PDF sudah ada. Jika folder tidak ada, maka akan dibuat. Fungsi ini akan mengembalikan path folder tersebut.

# Mengekstrak Barang Bukti

In [3]:
import re

def extract_barang_bukti(text):
    # Periksa apakah input berupa string
    if isinstance(text, str):
        # Memperbaiki regex untuk menangkap berbagai bentuk pengenalan barang bukti
        match = re.search(r'(?i)(Menetapkan barang bukti|Menyatakan barang bukti|Barang bukti berupa|yang diajukan ke persidangan ini(?: berupa)?)\s*[:：]?\s*(.*?)(?=\s*(?:Dirampas|Membebankan|Menyatakan|[;])|$|\n\d+\.\s|$)', text, re.DOTALL | re.IGNORECASE)
        if match:
            barang_bukti = match.group(2).strip()
            # Membersihkan kata "berupa" jika ada
            barang_bukti = re.sub(r'berupa\s*[:：]?\s*', '', barang_bukti)
            return barang_bukti.strip()
    return None

Fungsi `extract_barang_bukti` menggunakan ekspresi reguler untuk mengekstrak bagian dari teks yang berisi informasi mengenai 'Barang Bukti'. Ini mencari kata kunci "Barang Bukti" dan mengambil teks setelahnya.

# Mengambil Data Detail Putusan

In [4]:
# Fungsi untuk mengambil konten dari halaman detail putusan
def get_detail_data(detail_url, pdf_dir):
    detail_response = requests.get(detail_url)
    detail_soup = BeautifulSoup(detail_response.content, 'html.parser')

    # Mengambil tabel dengan class 'table'
    table = detail_soup.find('table', class_='table')

    if not table:
        print(f"Tabel tidak ditemukan di {detail_url}")
        return None

    # Mengambil semua <tr> di dalam tabel
    rows = table.find_all('tr')

    # Dictionary untuk menyimpan data putusan
    data = {
        'Nomor': None,
        'Lembaga Peradilan': None,
        'Catatan Amar': None,
        'File Path': None,  # Untuk menyimpan lokasi file PDF yang diunduh
        'Barang Bukti': None
    }

    for row in rows:
        # Mengambil semua <td> dalam <tr>
        td_elements = row.find_all('td')

        if len(td_elements) >= 2:  # Pastikan ada setidaknya dua elemen <td>
            header_text = td_elements[0].get_text(strip=True)  # Ambil teks dari <td> pertama
            value_text = td_elements[1].get_text(strip=True)   # Ambil teks dari <td> kedua

            # Menyimpan data ke dictionary sesuai dengan nama kolom
            if 'Nomor' in header_text:
                data['Nomor'] = value_text
            elif 'Lembaga Peradilan' in header_text:
                data['Lembaga Peradilan'] = value_text
            elif 'Catatan Amar' in header_text:
                data['Catatan Amar'] = value_text

    # Mengecek apakah nomor mengandung tahun 2022, 2023, atau 2024
    if data['Nomor'] and not re.search(r'\b(2022|2023|2024)\b', data['Nomor']):
        return None  # Abaikan jika tahun tidak sesuai

    # Ekstrak 'Barang Bukti' dari 'Catatan Amar' dengan cara lebih fleksibel
    data['Barang Bukti'] = extract_barang_bukti(data['Catatan Amar'])

    # Mencoba mengunduh PDF
    pdf_link = detail_soup.find('a', href=True, string=lambda x: x and x.endswith('.pdf'))
    if pdf_link:
        pdf_url = pdf_link['href']
        pdf_url = requests.compat.urljoin(detail_url, pdf_url)  # Buat URL PDF lengkap jika perlu
        safe_filename = re.sub(r'[\/:*?"<>|]', '_', data['Nomor'])  # Mengganti karakter yang tidak aman pada nama file
        pdf_path = os.path.join(pdf_dir, f"{safe_filename}.pdf")
        pdf_response = requests.get(pdf_url)

        if pdf_response.status_code == 200:
            # Menyimpan file PDF hanya jika pengunduhan berhasil
            with open(pdf_path, 'wb') as pdf_file:
                pdf_file.write(pdf_response.content)

            print(f"PDF berhasil diunduh: {pdf_path}")
            # Menyimpan path file PDF ke dalam data
            data['File Path'] = pdf_path
        else:
            print(f"Gagal mengunduh PDF dari {pdf_url} dengan status {pdf_response.status_code}")
            return None  # Batalkan pengambilan data jika PDF gagal diunduh
    else:
        print(f"Tidak ada link PDF ditemukan di {detail_url}")
        return None  # Batalkan pengambilan data jika tidak ada link PDF

    return data  # Hanya kembalikan data jika PDF berhasil diunduh

Fungsi `get_detail_data` digunakan untuk mengambil data dari halaman detail putusan. Ini mencakup ekstraksi informasi dari tabel HTML dan mencoba mengunduh file PDF terkait. Jika berhasil mengunduh PDF, path file disimpan dalam dictionary yang berisi informasi putusan.

# Mengemas PDF Menjadi ZIP

In [5]:
# Fungsi untuk mengemas folder PDF menjadi file ZIP
def create_zip_from_folder(folder_path, output_zip):
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))
    print(f"File ZIP berhasil dibuat: {output_zip}")

Fungsi `create_zip_from_folder` akan membuat file ZIP dari semua file dalam folder yang diberikan. Ini akan digunakan untuk mengemas file PDF yang diunduh ke dalam satu file ZIP.

# Mengambil dan Memproses Data dari Halaman Web

In [6]:
# URL halaman direktori utama
base_url = 'https://putusan3.mahkamahagung.go.id/direktori/index/pengadilan/pn-rantau-prapat/kategori/narkotika-dan-psikotropika-1/tahunjenis/putus/tahun/2024.html'
pdf_folder = create_path('putusan')  # Membuat folder untuk menyimpan file PDF

total_data = 0
putusan_list = []  # List untuk menyimpan data putusan
page_number = 1

# Variabel untuk menghitung statistik
total_halaman = 0
berhasil_diekraksi = 0
berhasil_pdf = 0
gagal_diekraksi = 0
gagal_pdf = 0

# Fungsi untuk memperbarui statistik
def update_statistics(detail_data, pdf_data):
    global total_halaman, berhasil_diekraksi, berhasil_pdf, gagal_diekraksi, gagal_pdf

    total_halaman += 1

    if detail_data:
        berhasil_diekraksi += 1  # Jika data berhasil diekstraksi
    else:
        gagal_diekraksi += 1  # Jika data gagal diekstraksi atau tidak sesuai

    if pdf_data:
        berhasil_pdf += 1  # Jika PDF berhasil diunduh
    else:
        gagal_pdf += 1  # Jika PDF gagal diunduh

# Looping untuk mengambil data dari halaman hingga mencapai 100 data
while total_data < 50:
    # Memperbarui URL untuk pagination
    if page_number > 1:
        page_url = f"https://putusan3.mahkamahagung.go.id/direktori/index/pengadilan/pn-rantau-prapat/kategori/narkotika-dan-psikotropika-1/tahunjenis/putus/tahun/2024/page/{page_number}"
    else:
        page_url = base_url  # Halaman pertama tetap URL dasar

    print(f"Mengambil data dari: {page_url}")

    # Mengirim request ke halaman direktori
    response = requests.get(page_url)

    # Cek apakah permintaan berhasil
    if response.status_code == 200:
        # Parsing konten HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Mencari semua tag <strong> di halaman
        strong_tags = soup.find_all('strong')

        # Mengambil link detail dari setiap strong tag
        for strong in strong_tags:
            link_tag = strong.find('a', href=True)  # Mencari link dalam <strong>
            if link_tag:
                detail_url = link_tag['href']  # Mendapatkan URL detail
                print(f"Mengambil data dari: {detail_url}")
                detail_data = get_detail_data(detail_url, pdf_folder)  # Ambil data dari halaman detail

                # Hanya memperbarui statistik jika detail_data tidak None
                if detail_data:
                    update_statistics(detail_data, detail_data.get('File Path') is not None)

                    if detail_data['Nomor'] not in [d['Nomor'] for d in putusan_list]:
                        putusan_list.append(detail_data)  # Simpan data ke list
                        total_data += 1  # Increment total data yang berhasil diambil

                # Hentikan jika sudah mencapai 100 data
                if total_data >= 100:
                    break
    else:
        print("Gagal mengakses halaman.")
        break  # Keluar dari loop jika gagal mengakses halaman

    page_number += 1  # Naik ke halaman berikutnya

# Menampilkan statistik setelah proses selesai
print(f"\nTotal halaman yang diproses: {total_halaman}")
print(f"Berhasil diekstraksi: {berhasil_diekraksi} ({(berhasil_diekraksi / total_halaman) * 100:.2f}%)")
print(f"Berhasil mengunduh PDF: {berhasil_pdf} ({(berhasil_pdf / total_halaman) * 100:.2f}%)")
print(f"Gagal diekstraksi atau tidak sesuai: {gagal_diekraksi} ({(gagal_diekraksi / total_halaman) * 100:.2f}%)")
print(f"Gagal mengunduh PDF: {gagal_pdf} ({(gagal_pdf / total_halaman) * 100:.2f}%)")
print("Proses crawling selesai.")

Output disimpan di putusan
Mengambil data dari: https://putusan3.mahkamahagung.go.id/direktori/index/pengadilan/pn-rantau-prapat/kategori/narkotika-dan-psikotropika-1/tahunjenis/putus/tahun/2024.html
Mengambil data dari: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefc18c5d4a9f42bba3303731363437.html
PDF berhasil diunduh: putusan/932_Pid.Sus_2024_PN Rap.pdf
Mengambil data dari: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefc18c41577c428060303731363030.html
PDF berhasil diunduh: putusan/943_Pid.Sus_2024_PN Rap.pdf
Mengambil data dari: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefc119e6a00b88b5af313733373235.html
PDF berhasil diunduh: putusan/946_Pid.Sus_2024_PN Rap.pdf
Mengambil data dari: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefbc852b161a128d3c323134323431.html
Tidak ada link PDF ditemukan di https://putusan3.mahkamahagung.go.id/direktori/putusan/zaefbc852b161a128d3c323134323431.html
Mengambil data dari: https://putusan3.mahkama

Pada bagian ini, kita mengakses halaman utama dan halaman selanjutnya untuk mengambil data putusan. Data putusan yang berhasil diambil disimpan dalam `putusan_list`. Setelah mencapai 50 data, proses pengambilan berhenti.

# Menyimpan Data ke CSV dan Mengemas PDF

In [7]:
# Membuat DataFrame untuk menyimpan data
df = pd.DataFrame(putusan_list)

# Menampilkan data
print(df)

# Menyimpan ke file CSV jika diperlukan
df.to_csv('data_putusan.csv', index=False)

# Menggunakan fungsi untuk membuat file ZIP
zip_path = 'putusan_files.zip'
create_zip_from_folder(pdf_folder, zip_path)

                      Nomor Lembaga Peradilan  \
0   932/Pid.Sus/2024/PN Rap                PN   
1   943/Pid.Sus/2024/PN Rap                PN   
2   946/Pid.Sus/2024/PN Rap                PN   
3   837/Pid.Sus/2024/PN Rap                PN   
4   953/Pid.Sus/2024/PN Rap                PN   
5   951/Pid.Sus/2024/PN Rap                PN   
6   870/Pid.Sus/2024/PN Rap                PN   
7   917/Pid.Sus/2024/PN Rap                PN   
8   871/Pid.Sus/2024/PN Rap                PN   
9   854/Pid.Sus/2024/PN Rap                PN   
10  874/Pid.Sus/2024/PN Rap                PN   
11  855/Pid.Sus/2024/PN Rap                PN   
12  817/Pid.Sus/2024/PN Rap                PN   
13  835/Pid.Sus/2024/PN Rap                PN   
14  834/Pid.Sus/2024/PN Rap                PN   
15  942/Pid.Sus/2024/PN Rap                PN   
16  841/Pid.Sus/2024/PN Rap                PN   
17  830/Pid.Sus/2024/PN Rap                PN   
18  842/Pid.Sus/2024/PN Rap                PN   
19  944/Pid.Sus/2024

Data yang berhasil diambil disimpan dalam DataFrame pandas dan kemudian disimpan ke file CSV. PDF yang diunduh juga dikemas dalam file ZIP.

# Mengekstrak PDF ke TXT

In [8]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.3 MB/s eta 0:00:00


In [9]:
import os
import re
import zipfile
from PyPDF2 import PdfReader

# Fungsi untuk membersihkan dan merapikan teks yang diekstrak
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Hilangkan whitespace yang tidak perlu
    text = re.sub(r'Direktori Putusan.*?putusan\.mahkamahagung\.go\.id', '', text, flags=re.IGNORECASE)  # Hilangkan watermark
    return text.strip()

# Fungsi untuk mengekstrak teks dari file PDF
def extract_pdf_to_txt(pdf_path, output_txt_path):
    try:
        reader = PdfReader(pdf_path)
        all_text = ""

        for page in reader.pages:
            raw_text = page.extract_text()
            cleaned_text = clean_text(raw_text)
            all_text += cleaned_text + "\n"

        with open(output_txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(all_text)

        print(f"Teks berhasil diekstrak ke: {output_txt_path}")
    except Exception as e:
        print(f"Terjadi kesalahan saat memproses file PDF: {e}")

# Fungsi untuk memproses semua file PDF dalam folder
def process_all_pdfs_in_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if file_name.lower().endswith('.pdf'):
            pdf_path = os.path.join(input_folder, file_name)
            output_txt_path = os.path.join(output_folder, f"{os.path.splitext(file_name)[0]}.txt")
            extract_pdf_to_txt(pdf_path, output_txt_path)

# Fungsi untuk membuat zip dari folder output
def zip_output_folder(output_folder, zip_path):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(output_folder):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, output_folder)
                zipf.write(file_path, arcname)
    print(f"Folder telah dikompres menjadi: {zip_path}")

# Fungsi untuk mengekstrak file ZIP
def extract_zip(input_zip_path, extract_to_folder):
    with zipfile.ZipFile(input_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
    print(f"File ZIP diekstrak ke: {extract_to_folder}")

# Contoh penggunaan
zip_input_path = "/content/putusan_files.zip"  # Path file ZIP
temp_extracted_folder = "/content/putusan_extracted"  # Folder sementara untuk file PDF
output_folder_path = "/content/txt_files"  # Folder output untuk file TXT
zip_output_path = "/content/txt_files.zip"  # Path untuk file ZIP hasil

# Ekstrak file ZIP
extract_zip(zip_input_path, temp_extracted_folder)

# Proses semua file PDF
process_all_pdfs_in_folder(temp_extracted_folder, output_folder_path)

# Kompres hasil TXT ke ZIP
zip_output_folder(output_folder_path, zip_output_path)

File ZIP diekstrak ke: /content/putusan_extracted
Teks berhasil diekstrak ke: /content/txt_files/799_Pid.Sus_2024_PN Rap.txt
Teks berhasil diekstrak ke: /content/txt_files/944_Pid.Sus_2024_PN Rap.txt
Teks berhasil diekstrak ke: /content/txt_files/840_Pid.Sus_2024_PN Rap.txt
Teks berhasil diekstrak ke: /content/txt_files/834_Pid.Sus_2024_PN Rap.txt
Teks berhasil diekstrak ke: /content/txt_files/725_Pid.Sus_2024_PN Rap.txt
Teks berhasil diekstrak ke: /content/txt_files/850_Pid.Sus_2024_PN Rap.txt
Teks berhasil diekstrak ke: /content/txt_files/946_Pid.Sus_2024_PN Rap.txt
Teks berhasil diekstrak ke: /content/txt_files/742_Pid.Sus_2024_PN Rap.txt
Teks berhasil diekstrak ke: /content/txt_files/845_Pid.Sus_2024_PN Rap.txt
Teks berhasil diekstrak ke: /content/txt_files/726_Pid.Sus_2024_PN Rap.txt
Teks berhasil diekstrak ke: /content/txt_files/814_Pid.Sus_2024_PN Rap.txt
Teks berhasil diekstrak ke: /content/txt_files/951_Pid.Sus_2024_PN Rap.txt
Teks berhasil diekstrak ke: /content/txt_files/874

# Menambahkan tokens

In [10]:
!pip install transformers

In [11]:
from transformers import pipeline

In [12]:
# Inisialisasi pipeline QA dengan model dan tokenizer Indobert
qa_pipeline = pipeline(
    "question-answering",
    model="Rifky/Indobert-QA",
    tokenizer="Rifky/Indobert-QA"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/532 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [13]:
# Ekstrak tokenizer dari pipeline
tokenizer = qa_pipeline.tokenizer

In [14]:
# Fungsi untuk menghitung token menggunakan tokenizer baru
def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

In [15]:
# Pastikan kolom tersebut ada dan proses penghitungan token
if 'Catatan Amar' in df.columns:
    # Isi nilai yang hilang
    df['Catatan Amar'] = df['Catatan Amar'].fillna('').astype(str)

    # Menerapkan fungsi count_tokens per baris
    df['Tokens'] = df['Catatan Amar'].apply(count_tokens)
else:
    print("Error: Column 'Catatan Amar' not found in DataFrame")

In [16]:
df

,Nomor,Lembaga Peradilan,Catatan Amar,File Path,Barang Bukti,Tokens
0,932/Pid.Sus/2024/PN Rap,PN,MENGADILI:Menyatakan TerdakwaNur Afifah Adila ...,putusan/932_Pid.Sus_2024_PN Rap.pdf,1 (satu) bungkus plastik klip transparan sedan...,231
1,943/Pid.Sus/2024/PN Rap,PN,MENGADILI:Menyatakan Terdakwa Rindu Efendy Gul...,putusan/943_Pid.Sus_2024_PN Rap.pdf,1 (satu) bungkus kertas warna cokelat berisi N...,188
2,946/Pid.Sus/2024/PN Rap,PN,MENGADILIMenyatakan Terdakwa Joni Pransiscus S...,putusan/946_Pid.Sus_2024_PN Rap.pdf,2 (dua) buah plastik klip transparan berisi kr...,306
3,837/Pid.Sus/2024/PN Rap,PN,MENGADILI :Menyatakan Terdakwa Abdullah Hajar ...,putusan/837_Pid.Sus_2024_PN Rap.pdf,18 (delapan belas) bungkus kertas warna putih ...,251
4,953/Pid.Sus/2024/PN Rap,PN,MENGADILI:Menyatakan TerdakwaIndra Rosana Siah...,putusan/953_Pid.Sus_2024_PN Rap.pdf,1 (satu) bungkus plastik klip berisi kristal p...,287
5,951/Pid.Sus/2024/PN Rap,PN,MENGADILI:Menyatakan TerdakwaFeri Silalahi ali...,putusan/951_Pid.Sus_2024_PN Rap.pdf,1 (satu) bungkus plastik klip tembus pandang b...,279
6,870/Pid.Sus/2024/PN Rap,PN,MENGADILI:Menyatakan Terdakwa Heri Purnomo Ali...,putusan/870_Pid.Sus_2024_PN Rap.pdf,1 (satu) bungkus plastik klip transparan beris...,343
7,917/Pid.Sus/2024/PN Rap,PN,MENGADILIMenyatakan Terdakwa Rio Saputra Munth...,putusan/917_Pid.Sus_2024_PN Rap.pdf,2 (dua) bungkus plastik klip kecil transparan ...,294
8,871/Pid.Sus/2024/PN Rap,PN,MENGADILIMenyatakan Terdakwa M. Ardiansyah Ali...,putusan/871_Pid.Sus_2024_PN Rap.pdf,1 (satu) bungkus plastik klip sedang berisi na...,223
9,854/Pid.Sus/2024/PN Rap,PN,MENGADILIMenyatakan Terdakwa I. Azmarizal Alia...,putusan/854_Pid.Sus_2024_PN Rap.pdf,1 (satu) bungkus plastik klip besar berisi Nar...,253


In [17]:
df_cleaned = df.dropna()

In [18]:
# Drop rows after the first 50
df = df_cleaned.iloc[:49]

In [19]:
df

,Nomor,Lembaga Peradilan,Catatan Amar,File Path,Barang Bukti,Tokens
0,932/Pid.Sus/2024/PN Rap,PN,MENGADILI:Menyatakan TerdakwaNur Afifah Adila ...,putusan/932_Pid.Sus_2024_PN Rap.pdf,1 (satu) bungkus plastik klip transparan sedan...,231
1,943/Pid.Sus/2024/PN Rap,PN,MENGADILI:Menyatakan Terdakwa Rindu Efendy Gul...,putusan/943_Pid.Sus_2024_PN Rap.pdf,1 (satu) bungkus kertas warna cokelat berisi N...,188
2,946/Pid.Sus/2024/PN Rap,PN,MENGADILIMenyatakan Terdakwa Joni Pransiscus S...,putusan/946_Pid.Sus_2024_PN Rap.pdf,2 (dua) buah plastik klip transparan berisi kr...,306
3,837/Pid.Sus/2024/PN Rap,PN,MENGADILI :Menyatakan Terdakwa Abdullah Hajar ...,putusan/837_Pid.Sus_2024_PN Rap.pdf,18 (delapan belas) bungkus kertas warna putih ...,251
4,953/Pid.Sus/2024/PN Rap,PN,MENGADILI:Menyatakan TerdakwaIndra Rosana Siah...,putusan/953_Pid.Sus_2024_PN Rap.pdf,1 (satu) bungkus plastik klip berisi kristal p...,287
5,951/Pid.Sus/2024/PN Rap,PN,MENGADILI:Menyatakan TerdakwaFeri Silalahi ali...,putusan/951_Pid.Sus_2024_PN Rap.pdf,1 (satu) bungkus plastik klip tembus pandang b...,279
6,870/Pid.Sus/2024/PN Rap,PN,MENGADILI:Menyatakan Terdakwa Heri Purnomo Ali...,putusan/870_Pid.Sus_2024_PN Rap.pdf,1 (satu) bungkus plastik klip transparan beris...,343
7,917/Pid.Sus/2024/PN Rap,PN,MENGADILIMenyatakan Terdakwa Rio Saputra Munth...,putusan/917_Pid.Sus_2024_PN Rap.pdf,2 (dua) bungkus plastik klip kecil transparan ...,294
8,871/Pid.Sus/2024/PN Rap,PN,MENGADILIMenyatakan Terdakwa M. Ardiansyah Ali...,putusan/871_Pid.Sus_2024_PN Rap.pdf,1 (satu) bungkus plastik klip sedang berisi na...,223
9,854/Pid.Sus/2024/PN Rap,PN,MENGADILIMenyatakan Terdakwa I. Azmarizal Alia...,putusan/854_Pid.Sus_2024_PN Rap.pdf,1 (satu) bungkus plastik klip besar berisi Nar...,253


In [20]:
# Menyimpan atau meninjau DataFrame yang telah diperbarui
df.to_csv('datasets.csv', index=False)
print("Token counts have been added and saved to 'datasets.csv'.")

Token counts have been added and saved to 'datasets.csv'.
